# Convolutional NN

In [ ]:
from __future__ import print_function
import numpy as np 
np.random.seed(1337) 
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.models import Model #for functional API
from keras.layers import Dense, Activation, Flatten, MaxPooling2D, Dropout, Input
from keras.layers import Conv2D #layer that makes our CNN 

In [ ]:
batch_size = 32 # 32 data points trained with model at 1 epoch
num_classes = 10 
epochs = 5 #how many times NN will see our data

In [ ]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() #already splited

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 #normalize
x_test /= 255

print('x_train shape:', x_train.shape) #50000 data points, each is 32*32*3 matrix, RGB 
print(x_train.shape[0], 'train samples') 
print(x_test.shape[0], 'test samples')

In [ ]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes) #convert labels into binary
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential() #load model

model.add(Conv2D(16, (3, 3), padding='same', #first class Conv2D, 16 filters, each 3 by 3 matrix
                 input_shape=x_train.shape[1:])) #shape of input (without shape[0] which is number of our data points)
model.add(Activation('relu')) # add activation function 

model.add(Conv2D(8, (3, 3))) #another Conv2D layer, input_shape argument is not needed anymore
model.add(Activation('relu')) 

model.add(MaxPooling2D(pool_size = (2,2))) # max pooling, split our value in 2*2 matrix, and get max(values in this pool) / Improve accuracy a little bit

model.add(Dropout(0.2)) #20% of pixels coming from the prev.layer will be dropped out
#This way the NN could generalize more and learn more efficient without even see the whole picture

model.add(Flatten()) #flatten! squize into 1-dim vector because dense layer needs something in 1 dimension
model.add(Dense(num_classes)) # need to add this one, to have our number of classes!! but works only with one dim, so added flattn
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6) #optimizer

# Let's train the model
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
hist1 = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

In [ ]:
# и все ;)  Натренировали и забили хуй

In [ ]:
hist1.history

# Functional API

In [ ]:
inputs = Input(shape=(32, 32, 3))

x = Conv2D(16, (3, 3), padding='same')(inputs) 
x = Activation('relu')(x) 

x = Conv2D(8, (3, 3))(x) 
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x) 

model = Model([inputs], output)

In [ ]:
model.summary()

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


hist2 = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

In [ ]:
hist2.history

## Inception Module / Graph model / Goggle Net Model

In [ ]:
# input layer is the same as our typical CNN model 
inputs = Input(shape=(32, 32, 3))

## ----------- New Stuff Starts Here --------- 


tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs) #tower like on the picture in my notes
#64 filter 1 by 1 each
# we pass inputs to this tower

tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu', name='t1_conv')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)


x = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)  #filter concatenation

## ----------- New Stuff Ends Here --------- 

# Rest of the model, again, remains the same 

x = Conv2D(8, (3, 3))(x)    
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x) 



In [ ]:
model = Model([inputs], output)
model.summary()  # Notice the 'Connected To' in the summary 

#first 3 are basis of the towers - connected to the input
#and so on

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


hist3 = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

In [ ]:
hist3.history

## Residual Connections

In [ ]:
# input layer is the same as our typical CNN model 
inputs = Input(shape=(32, 32, 3))

## ----------- New Stuff Starts Here --------- 

# 3x3 conv with 3 output channels (same as input channels)
# Necessary that we keep the number of channels the same so that the layers can be added 

y = Conv2D(3, (3, 3), padding='same')(inputs)

# This is the residual connection link. Notice that we are no longer sequential 
z = keras.layers.add([inputs, y])   # this returns x + y. 
#basically adding layer consisting of original input and output of previous layer/s 


## ----------- New Stuff Ends Here --------- 

# Rest of the model, again, remains the same 

x = Conv2D(8, (3, 3))(z)    # Notice the 'z'  
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x) 



In [ ]:
model = Model([inputs], output)
model.summary()  # Notice the 'Connected To' in the summary 

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))


# Save model (to avoid model.fit (=training)

In [ ]:
!pip install h5py --user  # to install h5py module (needed for saving models)

In [ ]:
import h5py  # to ensure we have this package installed 

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="checkpoints/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1, 
                             mode='max')

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          callbacks=[checkpoint])

In [ ]:
model.evaluate(x_test, y_test)   # produces the same results 

# Load model

##### Doing basically all the steps 
##### Defining the same model, with same shapes
##### But not fitting it (because training is costly)
##### just loading previously saved weights and evaluate (much faster than training)

In [ ]:
from __future__ import print_function
import numpy as np 
np.random.seed(1337) 
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.models import Model #for functional API
from keras.layers import Dense, Activation, Flatten, MaxPooling2D, Dropout, Input
from keras.layers import Conv2D #layer that makes our CNN 

In [ ]:
batch_size = 32 # 32 data points trained with model at 1 epoch
num_classes = 10 
epochs = 5 #how many times NN will see our data

In [ ]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() #already splited

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 #normalize
x_test /= 255

print('x_train shape:', x_train.shape) #50000 data points, each is 32*32*3 matrix, RGB 
print(x_train.shape[0], 'train samples') 
print(x_test.shape[0], 'test samples')

In [ ]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes) #convert labels into binary
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# input layer is the same as our typical CNN model 
inputs = Input(shape=(32, 32, 3))

## ----------- New Stuff Starts Here --------- 

# 3x3 conv with 3 output channels (same as input channels)
# Necessary that we keep the number of channels the same so that the layers can be added 

y = Conv2D(3, (3, 3), padding='same')(inputs)

# This is the residual connection link. Notice that we are no longer sequential 
z = keras.layers.add([inputs, y])   # this returns x + y. 
#basically adding layer consisting of original input and output of previous layer/s 


## ----------- New Stuff Ends Here --------- 

# Rest of the model, again, remains the same 

x = Conv2D(8, (3, 3))(z)    # Notice the 'z'  
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x) 



model = Model([inputs], output)

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [ ]:
# just create the model as before and then load weight 
filepath="checkpoints/weights-improvement-05-0.48.hdf5"
model.load_weights(filepath)   


In [ ]:
model.evaluate(x_test, y_test)   # produces the same results 